In [1]:
print("ok")

ok


In [6]:
import os 
os.chdir("../")
%pwd

'e:\\Hofstra_Assignements\\RAG_CHATBOT'

In [7]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [8]:
def load_pdf_file(data):
    loader = DirectoryLoader(data, glob = "*.pdf", loader_cls = PyPDFLoader)

    documents = loader.load()
    return documents

In [9]:
extracted_data =  load_pdf_file(data = 'Data/')

In [14]:
#split the data into Text Chunks
def text_split (extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks


In [15]:
text_chunks = text_split(extracted_data)
print("length of the text chunks:", len(text_chunks))

length of the text chunks: 5859


In [16]:
from langchain.embeddings import HuggingFaceEmbeddings

In [17]:

def download_hugging_face_embedding():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [18]:
embeddings = download_hugging_face_embedding()

C:\Users\harsh\AppData\Local\Temp\ipykernel_13996\1295865577.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
e:\Hofstra_Assignements\Python\Miniconda\envs\rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
query_results = embeddings.embed_query("Hello World")
print("length", len(query_results))

length 384


In [56]:
from dotenv import load_dotenv 
load_dotenv()

True

In [57]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [58]:
import os
os.environ["PINECONE_API_KEY"]= PINECONE_API_KEY
os.environ["OPENAI_API_KEY"]= OPENAI_API_KEY

In [31]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "medicalbot"
pc.create_index(
    name=index_name,
    dimension=384,    
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1",
        ),
)

{
    "name": "medicalbot",
    "metric": "cosine",
    "host": "medicalbot-hp6ehnw.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [ ]:
#Embed each chunk and upsert the embeddings into your Pinecone index,
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents= text_chunks,
    index_name="medicalbot",
    embedding=embeddings,
)

In [35]:
#Exsisting index 
from langchain_pinecone import PineconeVectorStore
docsearch= PineconeVectorStore.from_existing_index(
    index_name="medicalbot",
    embedding=embeddings
)

In [37]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [38]:
retriever_docs = retriever.invoke("what is Acne?")

In [39]:
retriever_docs

[Document(id='9639b8c5-9cdb-48b4-9a07-d31b1aab57fe', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='2c493578-1fa3-46af-b3cf-0435532f8e3f', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM 

In [87]:
#Not working
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [88]:
from langchain_ollama import OllamaLLM
llm = OllamaLLM(
    model="llama3",
    temperature=0.42,
    num_ctx=5000,                 # raise if you need bigger context
    base_url="http://localhost:11434"
)


In [89]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
system_prompt=(
    "You are an assistant for question-answering tasks."
    "Use the following pieces of retrived context to answer"
    "the question. If you don't know the answer , say that you don't know."
    "Use three senteces maxium and keoo the answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
    ("system", system_prompt),
    ("human", "{input}"),
    ]
)

In [90]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [91]:
response = rag_chain.invoke({"input": "what is Stats?"})
print(response["answer"])

I don't know. The provided context does not mention "Stats". It appears to be discussing blood counts and their purposes in medical exams.
